<a href="https://colab.research.google.com/github/drewlinsley/colabs/blob/master/dnn_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Python imports.**

Let's split this into separate blocks for clarity.

Here, you will import `drive` from the google colab library for connecting to [google drive](https://github.com/googlecolab/colabtools/blob/master/google/colab/drive.py) and `files` for [uploading local files](https://github.com/googlecolab/colabtools/blob/master/google/colab/files.py) (i.e. your machine) to this kernel. Finally, MediaFileUpload is a class for more efficient [uploads](https://github.com/googleapis/google-api-python-client/blob/master/googleapiclient/http.py). 

In [0]:
from google.colab import drive
from google.colab import files
from googleapiclient.http import MediaFileUpload


In [0]:
import numpy as np  # Note numpy is aliased as np
import Image
from glob import glob  # File path collection
import tensorflow as tf  # Note tensorflow is aliased as tf
from matplotlib import pyplot as plt  # Library for plotting images

# Keras model utilities
from keras.models import Model  # A Keras class for constructing a deep neural network model
from keras.models import Sequential  # A Keras class for connecting deep neural network layers 
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator  # A class for data loading during model training

# Keras ResNet routines
from keras.applications.resnet50 import ResNet50  # Import the ResNet deep neural network
from keras.preprocessing import image  # Routines for loading image data
from keras.applications.resnet50 import preprocess_input  # ResNet-specific routines for preprocessing images
from keras.applications.resnet50 import decode_predictions  # ResNet-specific routines for extracting predictions

# Keras layers
from keras.layers import Dense  # A fully connected neural networld layer
from keras.layers import Activation  # A class for point-wise nonlinearity layers
from keras.layers import Flatten  # Reshape a tensor into a matrix
from keras.layers import Dropout  # A regularization layer which randomly zeros neural network units during training.

# Optimizers
from keras.optimizers import Adam  # Adam optimizer https://arxiv.org/abs/1412.6980
from keras.optimizers import SGD  # Stochastic gradient descent optimizer

We will also install the google_images_download library to scrape images from websites. Later, we will pass these images through trained deep neural networks.

See how we can install libraries in python using the `pip` command. This is the python package manager, and is typically called with : `pip install my_package`. Note that because we are using the jupyter/ipython interface, we have to prepend an exclamation point `!` to call pip with the command line interpreter.

In [0]:
!pip install google_images_download
from google_images_download import google_images_download   #importing the library



**2. Set global variables**

This is not "good programming practice" under typical circumstances, but it is reasonable for python notebooks. So bear with me.

We will set global variables (paths, etc.) and mount google drive here.

In [0]:
TRAIN_DIR  = "/content/image_dataset"
drive.mount("/content/gdrive")
print("TensorFlow version: " + tf.__version__)

**3. Download images for your dnn.**

Using the google_images_download library, scrape images related to whatever you'd like. Later, we will pass these images through a deep neural network, and evaluate what it sees.

To scrape images, you will provide a python dictionary with key/value pairs that are interpreted by the library. These are (i) a comma-delimeted list of keywords of what image categories to scrape for, (ii) a per-category image limit, a boolean for whether or not to print progress, and an output directory. Feel free to change any of these.

Finally, we have inserted a few print commands that will 

In [0]:
response = google_images_download.googleimagesdownload()   #class instantiation
arguments = {
  "keywords":"cat,dog,bird,turtle,cheetah",
  "limit":10,
  "print_urls":False,
  "output_directory":TRAIN_DIR
}
paths = response.download(arguments)   #passing the arguments to the function
# print(paths)   #printing absolute paths of the downloaded images

categories = glob(TRAIN_DIR + '/*')
files = []
for cat in categories:
  files += glob(cat + '/*.jpg')
print files

**3. Load a pretrained ResNet dnn and process an image.**

We take advantage of Keras and use its built in ResNet50 model -- a near state-of-the-art 50-layer object classification model, that has been trained on millions of images. You can load this model by initializing the ResNet50 class with the weights-argument set to `imagenet` (the name of the large-image dataset used for training).

Since we aggregated paths of our scraped images in the variable `files` in the previous code block, we will begin by loading the first of those images into memory, then passing it into the model.

A few comments:



*   See how we have to specify `target_size` in the `load_img` method? This is because the ResNet50 model was trained on images of this size. The method will "center crop" images to this size after loading them.
*   After loading the image, you convert it into an array for processing by Keras and add a singleton dimension to the first axis. The resulting image tensor will have shape `batch/height/width/channels`, or 1/224/224/3. We have to do this because the model was trained on image batches (i.e., multiple images of 224/224/3 size).
*   Finally, the image is preprocessed by subtracting off the imagenet dataset mean value from its red/green/blue channels.  
*   The model has a 1000-dimension output, corresponding to the probability that the image belongs to any of 1000 categories that are in the imagenet dataset. The function decode_predictions returns the category names corresponding to the models predictions.



In [0]:
base_model = ResNet50(weights="imagenet")

img = image.load_img(files[0], target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = base_model.predict(x)
top_3 = decode_predictions(preds, top=3)[0]
print("Predicted: ", top_3)

f = plt.figure()
plt.imshow(img)
plt.grid('off')
plt.axis('off')
plt.title("Ground truth: %s\nprediction: %s" % (files[0], top_3))
plt.close(f)

**4. Initialize model and train with different sized images.**

We might want to take this model and retrain it on some other task.

This involves a few steps called model "fine tuning". We first reload the model but with the argument "include_top" set to False. This will load the model without its pretrained 1000-category classifier. The rest of the model uses convolutions -- sliding filters -- and by cutting off this top classifier we can map these filter responses to another task.

Note how here we also change the input image size. We can do this because the model is fully convolutional until the final layer, where there is a spatially global pooling, followed by a linear readout. See below:

![ResNet Architecture](https://cdn-images-1.medium.com/max/2100/1*S3TlG0XpQZSIpoDIUCQ0RQ.jpeg)

Compare the 34-layer residual network to the classic "VGG-19" network, which uses a cascade of fully connected layers to render decisions. 


In [0]:
HEIGHT     = 300
WIDTH      = 300

base_model = ResNet50(weights='imagenet', 
                      include_top=False, 
                      input_shape=(HEIGHT, WIDTH, 3))

**5. Augmentations**

CNNs are very sample inefficient: they need to be exposed to large amounts of image-level variability to reach their potential in image classification. This is because CNNs have very weak biases about natural images, relying only on convolutions that implement local-weight sharing.

Image datasets can be augmented with all sorts of transformations to expose CNNs to more variability and improve performance. You can do this easily in Keras with the built-in  [ImageDataGenerator class](https://keras.io/preprocessing/image/).

We set our batch size to 32 (the number of images we process at once during training), since this is a minimal size to use in a model that incorporates so-called "batch normalization" (like the ResNet).



In [0]:
BATCH_SIZE = 32

train_datagen =  ImageDataGenerator(
      preprocessing_function=preprocess_input,
      rotation_range=90,
      horizontal_flip=True,
      vertical_flip=True
    )

train_generator = train_datagen.flow_from_directory(TRAIN_DIR, 
                                                    target_size=(HEIGHT, WIDTH), 
                                                    batch_size=BATCH_SIZE)


** 6. "Finetune" a trained model for your task**
Now we put the pieces together and create the new model for fine tuning.

We build a function `build_fintune_model` that will add classifier layers (you can increase the number of these by adding numbers to the `FC_LAYERS` list) and prepare your model for finetuning. This also includes fixing the pretrained convolutional layers so that they can no longer be trained.

We also include dropout, which is an effective regularizer for CNNs.

In [0]:

def build_finetune_model(base_model, dropout, fc_layers, num_classes):
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)
    for fc in fc_layers:
        # New FC layer, random init
        x = Dense(fc, activation='relu')(x) 
        x = Dropout(dropout)(x)

    # New softmax layer
    predictions = Dense(num_classes, activation='softmax')(x) 
    
    finetune_model = Model(inputs=base_model.input, outputs=predictions)

    return finetune_model

class_list = ["cat", "dog", "bird", "turtle", "cheetah"]
FC_LAYERS  = [1024]
dropout    = 0.5

finetune_model = build_finetune_model(base_model, 
                                      dropout=dropout, 
                                      fc_layers=FC_LAYERS, 
                                      num_classes=len(class_list))

**7. Save performance and plot results.**

In [0]:

NUM_EPOCHS = 1
num_train_images = 100

adam = Adam(lr=0.00001)
finetune_model.compile(adam, loss='categorical_crossentropy', metrics=['accuracy'])

filepath="/content/gdrive/checkpoints/" + "ResNet50" + "_model_weights.h5"
checkpoint = ModelCheckpoint(filepath, monitor=["acc"], verbose=1, mode='max')
callbacks_list = [checkpoint]

history = finetune_model.fit_generator(train_generator, epochs=NUM_EPOCHS, workers=8, 
                                       steps_per_epoch=num_train_images // BATCH_SIZE, 
                                       shuffle=True, callbacks=callbacks_list)

plot_training(history)

# Plot the training and validation loss + accuracy
def plot_training(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(acc))

    plt.plot(epochs, acc, 'r.')
    plt.plot(epochs, val_acc, 'r')
    plt.title('Training and validation accuracy')

    # plt.figure()
    # plt.plot(epochs, loss, 'r.')
    # plt.plot(epochs, val_loss, 'r-')
    # plt.title('Training and validation loss')
    plt.show()

    plt.savefig('acc_vs_epochs.png')